# CMAES : Covariance Matrix Adaptation Evolutionary Strategy

Setup code and utility functions to plot and explore

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm 
from mpl_toolkits.mplot3d import axes3d
from numpy.random import multivariate_normal
import copy

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

try:
    import seaborn as sns
    sns.set_style("whitegrid")
    sns.set_context('talk')
    #sns.set(font_scale=1.4)
except ImportError:
    plt.style.use('seaborn-whitegrid')

In [ ]:
def range_from_bounds(bounds, resolution):
    (minx,miny),(maxx,maxy) = bounds
    x_range = np.arange(minx, maxx, (maxx-minx)/resolution)
    y_range = np.arange(miny, maxy, (maxy-miny)/resolution)    
    return x_range, y_range

def plot_problem_3d(problem, bounds, ax=None, resolution=100., 
                    cmap=cm.viridis_r, rstride=10, cstride=10, 
                    linewidth=0.15, alpha=0.65):
    """Plots a given benchmark problem in 3D mesh."""
    
    x_range, y_range = range_from_bounds(bounds, resolution=resolution)
    
    X, Y = np.meshgrid(x_range, y_range)
    Z = problem(X,Y)

    if not ax:
        fig = plt.figure(figsize=(11,6))
        ax = fig.gca(projection='3d')
        
    cset = ax.plot_surface(X, Y, Z, cmap=cmap, rstride=rstride, cstride=cstride, linewidth=linewidth, alpha=alpha)

In [ ]:
def plot_problem_contour(problem, bounds, optimum=None,
                          resolution=100., cmap=cm.viridis_r, 
                          alpha=0.45, ax=None):
    """Plots a given benchmark problem as a countour."""
    x_range, y_range = range_from_bounds(bounds, resolution=resolution)
    
    X, Y = np.meshgrid(x_range, y_range)
    Z = problem(X,Y)
    
    if not ax:
        fig = plt.figure(figsize=(6,6))
        ax = fig.gca()
        ax.set_aspect('equal')
        ax.autoscale(tight=True)
    
    cset = ax.contourf(X, Y, Z, cmap=cmap, alpha=alpha)
    
    if optimum:
        ax.plot(optimum[0], optimum[1], 'bx', linewidth=4, markersize=15)

In [ ]:
def plot_cov_ellipse(pos, cov, volume=.99, ax=None, fc='lightblue', ec='darkblue', alpha=1, lw=1):
    ''' Plots an ellipse that corresponds to a bivariate normal distribution.
    Adapted from http://www.nhsilbert.net/source/2014/06/bivariate-normal-ellipse-plotting-in-python/'''
    from scipy.stats import chi2
    from matplotlib.patches import Ellipse

    def eigsorted(cov):
        vals, vecs = np.linalg.eigh(cov)
        order = vals.argsort()[::-1]
        return vals[order], vecs[:,order]

    if ax is None:
        ax = plt.gca()

    vals, vecs = eigsorted(cov)
    theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))

    kwrg = {'facecolor':fc, 'edgecolor':ec, 'alpha':alpha, 'linewidth':lw}

    # Width and height are "full" widths, not radius
    width, height = 2 * np.sqrt(chi2.ppf(volume,2)) * np.sqrt(vals)
    ellip = Ellipse(xy=pos, width=width, height=height, angle=theta, **kwrg)
    ax.add_artist(ellip)

## Test functions

### Why benchmarks (test) functions?

In applied mathematics, [test functions](http://en.wikipedia.org/wiki/Test_functions_for_optimization), also known as artificial landscapes, are useful to evaluate characteristics of optimization algorithms, such as:

* Velocity of convergence.
* Precision.
* Robustness.
* General performance.

### [Bohachevsky benchmark problem](http://benchmarkfcns.xyz/benchmarkfcns/bohachevskyn2fcn.html)

$$\text{minimize } f(\mathbf{x}) = \sum_{i=1}^{N-1}(x_i^2 + 2x_{i+1}^2 - 0.3\cos(3\pi x_i) - 0.4\cos(4\pi x_{i+1}) + 0.7), \mathbf{x}\in \left[-100,100\right]^n,$$

> Optimum in $\mathbf{x}=\mathbf{0}$, $f(\mathbf{x})=0$.

In [ ]:
def _shifted_bohachevsky_impl(x, y, shift_x, shift_y):
    return (x-shift_x)**2 + 2.0 * (y-shift_y)**2 - 0.3 * np.cos(3.0 * np.pi * (x - shift_x)) - 0.4 * np.cos(4.0 * np.pi* (y - shift_y)) + 0.7
# def bohachevsky(x,y):
#     return x**2 + 2.0 * y**2 - 0.3 * np.cos(3.0 * np.pi * x) - 0.4 * np.cos(4.0 * np.pi*x) + 0.7

In [ ]:
from functools import partial
bohachevsky = partial(_shifted_bohachevsky_impl, shift_x = 0.0, shift_y = 0.0)
shifted_bohachevsky = partial(_shifted_bohachevsky_impl, shift_x = 2.0, shift_y = 2.0)

### [Griewank benchmark problem](http://benchmarkfcns.xyz/benchmarkfcns/griewankfcn.html)

$$\text{minimize } f(\mathbf{x}) = f(x_1, ..., x_n) = 1 + \sum_{i=1}^{n} \frac{x_i^{2}}{4000} - \prod_{i=1}^{n}\cos\left(\frac{2 \cdot x_i}{\sqrt{i}}\right)$$

> Optimum in $\mathbf{x}=\mathbf{0}$, $f(\mathbf{x})=0$.

In [ ]:
def _shifted_griewank_impl(x,y,shift_x, shift_y):
    return 1.0 + ((x-shift_x)**2 + (y-shift_y)**2) / 4000.0 - np.cos(2.0 * (x-shift_x)) * np.cos(2.0 * (y-shift_y) / np.sqrt(2.0))

# def griewank(x,y):
#     return 1.0 + (x**2 + y**2) / 4000.0 - np.cos(2.0 * x) * np.cos(2.0 * y / np.sqrt(2.0))

In [ ]:
griewank = partial(_shifted_griewank_impl, shift_x = 0.0, shift_y = 0.0)
shifted_griewank = partial(_shifted_griewank_impl, shift_x = 2.0, shift_y = 2.0)

In [ ]:
current_problem = bohachevsky

In [ ]:
plot_problem_3d(current_problem, ((-10,-10), (10,10)))

These problems has many local optima.

In [ ]:
plot_problem_3d(current_problem, ((-2.5,-2.5), (2.5,2.5)))

In [ ]:
ax = plt.figure(figsize=(8, 5)).gca()
plot_problem_contour(current_problem, ((-2.5,-2.5), (2.5,2.5)), optimum=(0,0), ax=ax)
ax.set_aspect('equal')

## Optimizing test functions using CMA-ES

### CMA-ES features

* Adaptation of the covariance matrix amounts to learning a second order model of the underlying objective function.
* This is similar to the approximation of the inverse Hessian matrix in the Quasi-Newton method in classical optimization. 
* In contrast to most classical methods, fewer assumptions on the nature of the underlying objective function are made. 
* *Only the ranking between candidate solutions is exploited* for learning the sample distribution and neither derivatives nor even the function values themselves are required by the method.


## Let's code up CMA from scratch!

Here are the steps of CMA in chronological order :
$$
\newcommand{\gv}[1]{\ensuremath{\mbox{\boldmath$ #1 $}}}
\newcommand{\bv}[1]{\ensuremath{\mathbf{#1}}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\order}[1]{\mathcal O \left( #1 \right)} % order of magnitude
$$

### Initialization
Set $ \mathbf{m} = \mathbf{0}, \mathbf{C} = \mathbf{I}, \sigma = 0.5, \mathbf{p}_c = \mathbf{0}, \mathbf{p}_{\sigma} = \mathbf{0} $

### Sampling
$$ \begin{aligned}
\mathbf{z}_{i} & \sim \mathcal{N}(\mathbf{0}, \mathbf{C}) \\
\mathbf{x}_{i} &= m+\sigma \mathbf{z}_{i}
\end{aligned} $$

### Selection and recombination
Sort the ppopulation by fitness to get $ \mu $  fit individuals
$$ \begin{aligned}
\langle\mathbf{z}\rangle_{w} &= \displaystyle\sum_{i=1}^{\mu} w_{i} \mathbf{z}_{i : \lambda} \\
\mathbf{m} &\longleftarrow \mathbf{m}+\sigma\langle\mathbf{z}\rangle_{w}
\end{aligned} $$

### Step size update
$$ \begin{aligned}
\mathbf{p}_{\sigma} &\longleftarrow\left(1-c_{\sigma}\right)
\mathbf{p}_{\sigma}+\sqrt{1-\left(1-c_{\sigma}\right)^{2}}
\sqrt{\frac{1}{\sum_{i=1}^{\mu} w_{i}^{2}}}
\mathbf{C}^{-\frac{1}{2}}\langle\mathbf{z}\rangle_{w} \\
\sigma &\longleftarrow \sigma
\exp{\left(\frac{c_{\sigma}}{d_{\sigma}}\left(\frac{\left\|p_{\sigma}\right\|}{E\|\mathcal{N}(\mathbf{0},
\mathbf{I})\|}-1\right)\right)} \\
\end{aligned} $$


### Covariance Matrix update
$$ \begin{aligned}
\mathbf{p}_{c} &\longleftarrow \left(1-c_{c}\right)
\mathbf{p}_{c}+\sqrt{1-\left(1-c_{c}\right)^{2}} \sqrt{\frac{1}{\sum_{i=1}^{\mu}
w_{i}^{2}}}\langle\mathbf{z}\rangle_{w} \\
\mathbf{Z} &= \sum_{i=1}^{\mu} w_{i} \mathbf{z}_{i : \lambda} \mathbf{z}_{i :
\lambda}^{T} \\
\mu_{c o v}&=\sqrt{\frac{1}{\sum_{i=1}^{\mu} w_{i}^{2}}} \\
\mathbf{C} &\longleftarrow\left(1-c_{c o v}\right) \mathbf{C}+c_{c o v}
\frac{1}{\mu_{c o v}} \mathbf{p}_{c} \mathbf{p}_{c}^{T}+c_{c o
v}\left(1-\frac{1}{\mu_{c o v}}\right) \mathbf{Z}
\end{aligned} $$

Some considerations:
 - `centroid` and `mean` are interchangeable.
 - `chi_N` is the expectation for the length of a random vector sampled from a multivariate normal distribution with $\mathbf{C} = \mathbf{I}$, and is used in the step-size update above. It can be analytically computed as $ \approx \sqrt{n} \left( 1 - \dfrac{1}{4n} + \dfrac{1}{21n^2} \right)$
 - `mu_eff` $ \mu_{\textrm{eff}} = \left(\displaystyle\sum_{i=1}^{\mu} w_{i}^{2}\right)^{-1} $ is the variance effective selection mass for the mean, as used in the CMA tutorial. Thus $\mu_{\textrm{cov}} = \sqrt{\mu_{\textrm{eff}}}$


In [ ]:
class CMAES:
    """Naive CMA implementation"""
    
    def __init__(self, initial_mean, sigma, popsize, **kwargs):
        """Please do all the initialization. The reserve space and 
        code for collecting the statistics are already provided."""
        self.dim = initial_mean.shape[0]
        
        # Things that evolve : centroid, sigma, paths etc.
        self.centroid = initial_mean
        self.sigma = sigma

        # pc is the path taken by the covariance matrix
        self.pc = """fill"""

        # ps is the path taken by sigma / step-size updates
        self.ps = np.zeros((self.dim, ))
        
        self.C = np.eye(self.dim)
        self.diagD, self.B = np.linalg.eigh(self.C)
    
        # Population size etc. 
        self.popsize = popsize
        self.mu = int((popsize / 2))
        
        # Update weights
        self.weights = np.arange(1, self.mu)[::-1]
        self.weights = self.weights / np.sum(self.weights)

        # Utility variables
        self.dim = initial_mean.shape[0]

        # Expectation of a normal distribution
        self.chiN = np.sqrt(self.dim) * (1.0 - 0.25 / self.dim + 1.0/(21.0 * self.dim**2))
        self.mueff = 1.0 / np.sum(self.weights**2)
        self.mucov = np.sqrt(self.mueff)
        self.generations = 0
        
        # Options
 
        # Sigma adaptation
        # cs is short for c_sigma
        self.cs = (self.mucov ** 2 + 2) / ((self.mucov ** 2 + 5) + self.dim)
        # ds is short for d_sigma
        factor = np.sqrt((self.mucov ** 2 - 1.0) / (self.dim + 1.0))
        self.ds = 1.0 + 2.0 * max(0.0, factor - 1) + self.cs
        
        # Covariance adaptation
        self.cc = """fill"""
        self.ccov = """fill"""
        # If implementing the latest version of CMA according to the tutorial, 
        # these parameters can be useful, if not that avoid
        self.ccov1 = 0.0
        self.ccovmu = 0.0
        
        ### Asserts to guide you on your paths
        #                    .--.   
        #  ::\`--._,'.::.`._.--'/::     Do or do not.
        #  ::::.  ` __::__ '  .::::    There is no try.
        #  ::::::-:.`'..`'.:-::::::
        #  ::::::::\ `--' /::::::::              -Yoda
                                
        assert self.dim == 2, "We are dealing with a two-dimensional problem only"
        assert self.centroid.shape == (2,), "Centroid shape is incorrect, did you tranpose it by mistake?"
        assert self.sigma > 0.0, "Sigma is not a non-zero positive number!"
        assert self.pc.shape == (2, ), "pc shape is incorrect, did you tranpose it by mistake?"
        assert self.ps.shape == (2, ), "ps shape is incorrect, did you tranpose it by mistake?"
        assert self.C.shape == (2, 2), "C's shape is incorrect, remember C is a matrix!"     
        assert type(self.popsize) == int, "Population size not an integer"
        assert self.popsize > 0 , "Population size is negative!"
        assert self.popsize > 2 , "Too little population size, make it >2"
        
        # Collect useful statistics 
        self.stats_centroids = []
        self.stats_new_centroids = []
        self.stats_covs = []
        self.stats_new_covs = []
        self.stats_offspring = []
        self.stats_offspring_weights = []
        self.stats_ps = []
    
    def run(self, problem):       
        while (# fill in your termination criterion here):
            # Sample the population here!
            # Its convenient to do it as a list of members
            # z_i
            population = list(np.random.multivariate_normal(np.array([0.0, 0.0]), self.C, (self.popsize, )))
            # self.C
            # self.centroid
            
            # Pass the population to update, which computes all new parameters
            # while sorting the populatoin
            self.update(problem, population)
            
            # increment generation counter
            self.generations += 1
        else:
            # returns the best individual at the last generation
            return population[0]

    def update(self, problem, population):
        """Update the current covariance matrix strategy from the
        *population*.
        
        :param population: A list of individuals from which to update the
                           parameters.
        """
        # -- store current state of the algorithm
        self.stats_centroids.append(copy.deepcopy(self.centroid))
        self.stats_covs.append(copy.deepcopy(self.C))
        
        # Sort the population here and work with only the sorted population
        x_i = self.centroid + self.sigma * np.array(population)
        """FILL : Python code to sort population goes here"""

        # new_population = sorted(x_i, key = problem)
        f_i = problem(x_i)
        idx = np.argsort(f_i)
        population = np.array(population)[idx]
 
        best_population = population[:self.mu]
            
        # -- store sorted offspring
        self.stats_offspring.append(copy.deepcopy(population))
        
        # Store old centroid in-case
        old_centroid = self.centroid
        # Update centroid to self.centroid here
        z_w = np.dot(self.weights, best_population)
        self.centroid = self.centroid + self.sigma * z_w
        
        # -- store new centroid
        self.stats_new_centroids.append(copy.deepcopy(self.centroid))
        
        # Cumulation : update evolution path
        # Remember to use self.B, self.diagD wihch we store later
        # See line 142-145 
        coeff = np.sqrt(1.0 - (1.0 - self.cs)**2)
        invsqrtC = self.B @ np.diag(1.0 / self.diagD) @ self.B.T
        self.ps = (1 - self.cs) * self.ps + coeff * self.mucov * (invsqrtC @ z_w)
        
        # -- store new evol path
        self.stats_ps.append(copy.deepcopy(self.ps))
        
        # Cumulation : update evolution path for centroid
        factor = np.sqrt(1.0 - ((1.0 - self.cc))**2)
        self.pc = (1.0 - self.cc) * self.pc + factor * self.mucov * z_w
        
        # Update covariance matrix
        Z = np.dot(self.weights, np.outer(z_i, z_i))
        
        self.C = """FILL : Code to calculate new covariance matrix """
        
        # -- store new covs
        self.stats_new_covs.append(copy.deepcopy(self.C))
        
        # Update new sigma in-place, can be done before too
        self.sigma *= np.exp(self.cs / self.ds * (np.linalg.norm(self.ps) / chiN) - 1.0)
        
        # Get the eigen decomposition for the covariance matrix to calculate inverse            
        diagD_squared, self.B = np.linalg.eigh(self.C)
        
        self.diagD = np.sqrt(diagD_squared)
        self.B = self.B
                
    def reset(self):
        """Clears everything to rerun the problem""" 
        pass

In [ ]:
initial_centroid = np.random.randn(2, )
cma_es = CMAES(initial_centroid, 0.2, 10)

In [ ]:
cma_es.run(current_problem)

### Visualizing CMA-ES progress 

First some setup code. This visualizes the progress of CMA based on the data we recorded in the class above and plots it in the objective function manifold.

In [ ]:
normalizer = colors.Normalize(vmin=np.min(cma_es.weights), vmax=np.max(cma_es.weights))
sm = cm.ScalarMappable(norm=normalizer, cmap=plt.get_cmap('gray'))

In [ ]:
from matplotlib import animation
from IPython.display import HTML

In [ ]:
def animate_cma_es(gen):
    ax.cla()
    plot_problem_contour(current_problem, ((-11,-11), (11,11)), optimum=(0,0), ax=ax)
    
    plot_cov_ellipse(cma_es.stats_centroids[gen], cma_es.stats_covs[gen], volume=0.99, alpha=0.29, 
                     fc='red', ec='darkred',
                     ax=ax)
    ax.plot(cma_es.stats_centroids[gen][0], cma_es.stats_centroids[gen][1], 'ro', markeredgecolor = 'none', ms=10)
    
    plot_cov_ellipse(cma_es.stats_new_centroids[gen], cma_es.stats_new_covs[gen], volume=0.99, 
                     alpha=0.29, fc='green', ec='darkgreen', ax=ax)
    ax.plot(cma_es.stats_new_centroids[gen][0], cma_es.stats_new_centroids[gen][1], 'go', markeredgecolor = 'none', ms=10)
    
    for i in range(gen+1):
        if i == 0:
            ax.plot((0,cma_es.stats_ps[i][0]),
                     (0,cma_es.stats_ps[i][1]), 'b--')
        else:
            ax.plot((cma_es.stats_ps[i-1][0],cma_es.stats_ps[i][0]),
                     (cma_es.stats_ps[i-1][1],cma_es.stats_ps[i][1]),'b--')
            
    for i,ind in enumerate(cma_es.stats_offspring[gen]):
        if i < len(cma_es.weights):
            color = sm.to_rgba(cma_es.weights[i])
        else:
            color= sm.to_rgba(normalizer.vmin)
        ax.plot(ind[0], ind[1], 'o', color = color, ms=5, markeredgecolor = 'none')
    
    ax.set_ylim((-10,10))
    ax.set_xlim((-10,10))
    ax.set_title('$generation=$' +str(gen))
    return []

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.gca()
anim = animation.FuncAnimation(fig, animate_cma_es, frames=cma_es.generations, interval=300, blit=True)
plt.close()

In the animation below :
* Current centroid and covariance: **red**.
* Updated centroid and covariance: **green**. 
* Sampled individuals: **shades of gray representing their corresponding weight**. (White is best)
* Evolution path: **blue line starting in (0,0)**. 

In [ ]:
HTML(anim.to_html5_video())